<a href="https://colab.research.google.com/github/jeong2624/Deep-Learning-for-the-Life-Sciences-coding-transcription/blob/main/Chapter5_Biophysics_and_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Biophysics and Machine Learning**

해당 Chapter에서는 생물물리학 (Biophysics)에 딥러닝을 활용하는 방법을 살펴봄.

특히 약물이 체내에 들어가서 생체 내 단백질과 어떤 결합을 하는지를 예측하는 문제에 집중.

약물과 단백질의 상호작용은 신약 개발에서 중요한 관심사이며, 단백질의 기능을 조절하면 치료 효과를 낼 수 있기 때문임.

따라서, 해당 Chapter에서는 특정 분자가 목적 단백질과 형성하는 상호작용을 예측하는 머신러닝 모델을 만드는 것이 목표!

In [1]:
# 필요한 라이브러리 설치
!pip install --pre deepchem
!pip install numpy torch_geometric pytorch_lightning dm-haiku
!pip install nglview

In [2]:
# 필요한 라이브러리 불러오기
import deepchem as dc
import numpy as np

# 재현성을 위해 시드값 배정
np.random.seed(20231110)

### **생물물리학 데이터 사례 연구**

In [3]:
# 분석의 편의를 위해 다음과 같이 RdkitGridFeaturizer 객체 만들기
# 그리드 피처화 : 단백질-리간드 결합에 대한 데이터를 피처화를 통해 벡터로 변환
grid = dc.feat.RdkitGridFeaturizer(
    voxel_width = 2.0, # 그리드를 구성하는 복셀 (Voxel, 부피 (volume)와 픽셀 (pixel)을 조합한 혼성어)의 크기를 2Å (옹스트롬)으로 정의
    feature_types = ['hbond', 'salt_bridge', 'pi_stack', 'cation_pi', 'ecfp', 'splif'], # 유용한 화학적 특성 : 수소결합, 이온결합, 파이 겹침 결합, ECFP (화학 지문), SPLIF (화학 지문)
    sanitize = True, # 단백질 3차원 구조는 종종 잘못된 형식이 포함돼 있기 때문에 해당 옵션을 통해 DeepChem이 탐지한 명백한 오류를 수정.
    flatten = True # 각각의 입력 구조에 대해 1차원 feature 벡터로 출력
)

In [4]:
# PDBBind 데이터셋 불러오기
# 해당 데이터셋은 많은 수의 생체 분자 (단백질, 핵산, 지질 등) 의 3차원 구조와 이들의 결합 친화력 (Binding affinity)를 포함하고 있음.
# 해당 데이터셋은 PDB (Protein Data Bank)에서 수집됨.

tasks, datasets, transformers = dc.molnet.load_pdbbind(
    featurizer = grid, # MoleculeNet이 자동으로 피쳐화 수행함.
    splitter = "random", # 무작위로 데이터 분할
    subset = "core" # PDB 데이터는 실험에 대한 설정과 측정값에 실험적인 차이가 있기 때문에 결과적으로 데이터 품질이 일정하지 못함. 따라서 정리된 핵심 데이터셋만 사용.
)

# Train, Validation, Test 데이터셋으로 분리
train_dataset, valid_dataset, test_dataset = datasets

#### **0. PDB 파일 시각화**

[NGLview](https://github.com/nglviewer/nglview) 라이브러리를 사용해 '2D3U' 단백질-리간드 복합체 시각화

In [5]:
from google.colab import output
output.enable_custom_widget_manager()
from IPython.display import display
import ipywidgets as widgets
import requests
import nglview as nv

# PDB 파일 불러오기
view = nv.show_file("https://raw.githubusercontent.com/deepchem/DeepLearningLifeSciences/master/Chapter05/2d3u.pdb")

# "2D3U" 단백질-리간드 시각화
view.representations = [
    {"type": "cartoon", "params": {
        "sele": "protein", "color": "residueindex"
    }},
    {"type": "ball+stick", "params": {
        "sele": "ligand"
    }}
]

view

NGLWidget()

#### **1. Random Forest 모델**

In [6]:
# 단백질-리간드 결합을 예측하는 머신러닝 모델 만들기 (Random Forest)
from sklearn.ensemble import RandomForestRegressor

sklearn_model = RandomForestRegressor(n_estimators = 100)
model = dc.models.SklearnModel(sklearn_model)

# 모델 학습
model.fit(train_dataset)

In [7]:
# 모델 성능 평가 (평균제곱오차, MSE)
metric = dc.metrics.Metric(dc.metrics.mean_squared_error)
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)

print("Train scores")
print(train_scores)

print("Test scores")
print(test_scores)

Train scores
{'mean_squared_error': 5.48236179136448}
Test scores
{'mean_squared_error': 2.8030007102378147}


#### **2. 다층 퍼셉트론**

In [8]:
# 단백질-리간드 결합을 예측하는 머신러닝 모델 만들기 (Multi-layer Perceptron)
n_features = train_dataset.X.shape[1]

nn_model = dc.models.MultitaskRegressor(
    n_tasks = len(tasks), # 예측하고자 하는 값의 개수
    n_features = n_features, # 변수의 개수
    layer_sizes = [2000, 1000], # Hidden layer의 너비를 각각 2,000과 1,000으로 설정
    dropout = 0.5, # 모델 과적합을 피하기 위해 50% dropout을 설정.
    learning_rate = 0.0003, # 학습률
)

# 모델 학습
nn_model.fit(train_dataset, nb_epoch = 50)

1.259886703491211

In [9]:
# 모델 성능 평가 (평균제곱오차, MSE)
metric = dc.metrics.Metric(dc.metrics.mean_squared_error)
train_scores = nn_model.evaluate(train_dataset, [metric], transformers)
test_scores = nn_model.evaluate(test_dataset, [metric], transformers)

print("Train scores")
print(train_scores)

print("Test scores")
print(test_scores)

Train scores
{'mean_squared_error': 5.5922429809524585}
Test scores
{'mean_squared_error': 3.076108678156255}
